In [1]:
import os
from athina.evals import DoesResponseAnswerQuery, ContextContainsEnoughInformation, Faithfulness
from athina.loaders import Loader
from athina.keys import AthinaApiKey, OpenAiApiKey
from athina.runner.run import EvalRunner
from athina.datasets import yc_query_mini
from athina.interfaces.athina import AthinaExperiment
import pandas as pd

from dotenv import load_dotenv
load_dotenv()

OpenAiApiKey.set_key(os.getenv('OPENAI_API_KEY'))
AthinaApiKey.set_key(os.getenv('ATHINA_API_KEY'))

/Users/akshat_g/athina/repos/athina-evals/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/akshat_g/athina/repos/athina-evals/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Create batch dataset from list of dict objects
raw_data = yc_query_mini.data

dataset = Loader().load_dict(raw_data)
pd.DataFrame(dataset)

,query,context,response,expected_response
0,What are some successful companies that went t...,[Y Combinator has invested in companies in var...,"Airbnb, Dropbox, Stripe, Reddit, Coinbase, Ins...",Airbnb and Stripe are 2 of the successful comp...
1,In which city is YC located?,"[Y Combinator is located in Mountain View, Cal...",Y Combinator is located in San Francisco,"YC is located in Mountain View, California."
2,How much equity does YC take?,[Y Combinator invests $500k in 200 startups tw...,YC invests $150k for 7%.,I cannot answer this question as I do not have...
3,How much equity does YC take?,[Y Combinator invests $500k in 200 startups tw...,I cannot answer this question as I do not have...,I cannot answer this question as I do not have...
4,Who founded YC and when was it founded?,[Y Combinator was founded in March 2005 by Pau...,Y Combinator was founded in 2005,Y Combinator was founded in March 2005 by Paul...
5,Does Y Combinator invest in startups outside t...,[Y Combinator invests in startups from all ove...,"Yes, Y Combinator invests in international sta...","Yes, Y Combinator invests in startups from all..."
6,How much does YC invest in startups?,[YC invests $150k for 7%.],$150k,YC invests $150k for 7%.
7,What is YC's motto?,[Y Combinator's motto is 'Make something peopl...,Make something people want,Make something people want


In [3]:
# Run the eval suite
eval_model = "gpt-4"

# Record an experiment
experiment = AthinaExperiment(
    experiment_name="yc-question-chatbot",
    experiment_description="YC Q&A with RAG model and few-shot examples",
    language_model_provider="openai",
    language_model_id="gpt-3.5-turbo",
    prompt_template=[],
    dataset_name="yc_dataset_mini",
)

In [4]:
# Define your evaluation suite
eval_suite = [
    DoesResponseAnswerQuery(model=eval_model),
    Faithfulness(model=eval_model),
    ContextContainsEnoughInformation(model=eval_model),
]

# Run the evaluation suite
batch_eval_result = EvalRunner.run_suite(
    evals=eval_suite,
    data=dataset,
    experiment=experiment,
    max_parallel_evals=2
)

batch_eval_result

201
You can view the evaluation results at https://app.athina.ai/develop/request/511c971b-ec94-4c30-9248-d28606ab44f7


,query,context,response,expected_response,Does Response Answer Query passed,Faithfulness passed,Context Contains Enough Information passed
0,What are some successful companies that went t...,[Y Combinator has invested in companies in var...,"Airbnb, Dropbox, Stripe, Reddit, Coinbase, Ins...",Airbnb and Stripe are 2 of the successful comp...,1.0,0.0,0.0
1,In which city is YC located?,"[Y Combinator is located in Mountain View, Cal...",Y Combinator is located in San Francisco,"YC is located in Mountain View, California.",1.0,0.0,1.0
2,How much equity does YC take?,[Y Combinator invests $500k in 200 startups tw...,YC invests $150k for 7%.,I cannot answer this question as I do not have...,1.0,0.0,0.0
3,How much equity does YC take?,[Y Combinator invests $500k in 200 startups tw...,I cannot answer this question as I do not have...,I cannot answer this question as I do not have...,0.0,0.0,0.0
4,Who founded YC and when was it founded?,[Y Combinator was founded in March 2005 by Pau...,Y Combinator was founded in 2005,Y Combinator was founded in March 2005 by Paul...,0.0,1.0,1.0
5,Does Y Combinator invest in startups outside t...,[Y Combinator invests in startups from all ove...,"Yes, Y Combinator invests in international sta...","Yes, Y Combinator invests in startups from all...",1.0,1.0,1.0
6,How much does YC invest in startups?,[YC invests $150k for 7%.],$150k,YC invests $150k for 7%.,1.0,1.0,1.0
7,What is YC's motto?,[Y Combinator's motto is 'Make something peopl...,Make something people want,Make something people want,1.0,1.0,1.0
